###### December 2019 - Roger Melko

Diagonalizing the Hamiltonian matrix for the transverse field Ising model to find the energy eigenvalues and eigenkets.   Calculate the groundstate magnetization.

We will use the same Hamiltonian convention as the QMC program:
$$
H = -J\sum_{\langle i j \rangle} \sigma^z_i \sigma^z_j - B \sum_i \sigma^x_i
$$
where ${\bf \sigma}_i$ are Pauli operators.  In this convention, the 1+1 CFT is at $h/J = 1$.

In [1]:
N = 6
Dim = 2^N

J = 1. #this is the exchange
B = 1. #this is the transverse field

Hamiltonian = zeros(Float32,Dim,Dim)   #This is your 2D Hamiltonian matrix

for Ket = 0:Dim-1  #Loop over Hilbert Space
    Diagonal = 0.
    for SpinIndex = 0:N-2  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        NextIndex = SpinIndex + 1
        Spin2 = 2*((Ket>>NextIndex)&1) - 1
        Diagonal = Diagonal - J*Spin1*Spin2 #spins are +1 and -1
    end
    Hamiltonian[Ket+1,Ket+1] = Diagonal
    
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1,Ket+1] = -B
    end
    
end

In the Julia LinearAlgebra package, the eigen function finds eigenvalues and eigenvectors.  They are ordered; i.e. the groundstate energy corresponds to index 1

In [2]:
using LinearAlgebra
Diag = eigen(Hamiltonian)     #Diagonalize the Hamiltonian
println(Diag.values)         #This is all of the eigenvalues
#index = findmin(Diag.values)  #The minimum eigenvalue corresponds to index=1
#println(index[1])
GroundState = Diag.vectors[:, 1]  #this gives the groundstate eigenvector
println(GroundState)

Float32[-7.296233, -6.8140793, -5.8778143, -5.3956623, -5.0239687, -4.5418262, -4.302187, -3.8200388, -3.754406, -3.6055527, -3.412465, -3.2722578, -3.1234045, -2.930317, -2.883771, -2.4016228, -2.3359861, -2.0299273, -1.9940414, -1.8538399, -1.547781, -1.5118952, -1.4821463, -1.1402044, -1.0, -0.7603617, -0.6580553, -0.61150837, -0.41841745, -0.2782154, -0.12936163, -0.06372833, 0.06372833, 0.12936258, 0.27821493, 0.4184208, 0.6115098, 0.65805674, 0.76036406, 1.000001, 1.1402035, 1.4821467, 1.5118971, 1.547781, 1.8538384, 1.9940424, 2.0299277, 2.3359857, 2.4016213, 2.88377, 2.9303164, 3.1234045, 3.2722597, 3.412462, 3.6055515, 3.7544053, 3.82004, 4.302188, 4.5418243, 5.023971, 5.3956633, 5.8778105, 6.814084, 7.29623]
Float32[-0.40054974, -0.20626862, -0.13093561, -0.16620174, -0.12267257, -0.074823074, -0.09524678, -0.15654245, -0.12267257, -0.06708649, -0.04720509, -0.06761941, -0.08746394, -0.06156687, -0.09110713, -0.1662011, -0.1309356, -0.069541946, -0.04565585, -0.061566886, -0.

In [3]:
##### Calculate the groundstate magnetization <m^2> in the Z direction
magnetization = 0
for Ket = 0:Dim-1  #Loop over Hilbert Space
    SumSz = 0.
    for SpinIndex = 0:N-1  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        SumSz += Spin1 #spin is +1 or -1
        #print(Spin1," ")
    end
    #println(SumSz," ",GroundState[Ket+1])
    magnetization += SumSz*SumSz*GroundState[Ket+1]^2  #Don't forgot to square the coefficients...
end
println(magnetization/(N*N))    

0.4824365366560717
